### Importing necessary libraries

In [ ]:
import os

In [ ]:
import numpy as np

In [ ]:
import glob
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import tensorflow
import seaborn as sns
import cv2
import imutils
from math import dist
import pandas as pd
%matplotlib inline

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

### Change the location to the directory where you cloned yolov5 originally
Link to clone Yolov5 repository: https://github.com/ultralytics/yolov5

In [ ]:
cd /home/abhinav/newyolo/yolov5/

### Running best trained YOLOv5 detection of center on images in image directory "/home/abhinav/M35/", saving results and detection coordinates in "/home/abhinav/M35/Results"

In [ ]:
!python detect.py --weights /homeHDD/pgc/yolov5_center/yolov5/runs/train/exp31/weights/best.pt --img 640 --conf 0.6 --source /home/abhinav/M35/ --project /home/abhinav/M35/ --name Results --save-txt
#2 changes to be made: Change the path to source folder where images are stored, and other path to save the results(most preferably in same folder)

In [ ]:
os.chdir("/home/abhinav/M35/Results/") #Change the path to where the results of previous code were saved

### The results file include text files for co-ordinates of center for each image. Marking those Co-ordinates in the original images

In [ ]:
dict1={}
for name in os.listdir():
    if ".jpg" in name:
        try:
            mask = cv2.imread(name)
            (h, w) = mask.shape[:2]
            txtname=name.split('.jpg')[0]+".txt"
            filename=open("./labels/"+txtname,'r')
            lines=filename.readlines()
            (x,y)=((lines[0].split(" "))[1],(lines[0].split(" "))[2])
            point=(int(float(x)*w),int(float(y)*h))
            dict1[name]=[point[0],point[1]]
            cv2.circle(mask, point, 7, (255, 0, 0), -1)
            cv2.imwrite(name,mask)
            img_orig = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
            plt.imshow(img_orig)
            plt.show()
        except:
            pass

In [ ]:
os.chdir("/home/abhinav/M35/Results/") #Change the path to the folder of images

### Using HoughCircles to draw a circle present in each image and determine the actual center, and print out the distance of the co-ordinates found for center using HoughCircle and co-ordinates of circle determined by ML module in previous lines of code

In [ ]:
dict2={}

output_name = []
output_distance = []
image_offset_notfound=[]
radius=[]

output_dir = "offset_result_contors"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
for name in os.listdir():
    if ".jpg" in name:
            mask = cv2.imread(name)
            gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
            blurred = cv2.medianBlur(gray, 5)
            circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, 150, param1=100, param2=50, minRadius=90, maxRadius=110)
            if circles is not None:
                detected_circles = np.uint16(np.around(circles))
                for (x,y,r) in detected_circles[0,:]:
                    cv2.circle(mask, (x,y), r, (0,255,0), 3)
                    cv2.circle(mask, (x,y), 3, (0,255,0), 5)
                    print("Center (x,y): ({}, {}) radius: {}".format(x,y,r))
                    dict2[name]=[[x,y],r]
            dist_value="Not found"
            try:
                dist_value=dist(dict1[name],dict2[name][0])
            except:
                pass
            img_orig = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
            plt.title("Distance: "+str(dist_value))
            print("{}:    {}".format(name, dist_value))
            if type(dist_value)==float:
                output_name.append(name)
                output_distance.append(dist_value)
                radius.append(r)
            else:
                image_offset_notfound.append(name)
            plt.imshow(img_orig)
            plt.savefig(f"offset_result_contors/M35_{name}.pdf")
            plt.show()
                
                

print(np.mean(radius))
#print(np.mean(dist_value))

In [ ]:
### Filtering dataset of both the dictionaries and making sure they have same images and labels

In [ ]:
keys_to_drop=[]
for key in dict2:
    if key not in dict1:
        keys_to_drop.append(key)
keys_to_drop
for key in keys_to_drop:
    if key in dict2:
        del dict2[key]
len(dict2)

In [ ]:
keys_to_drop=[]
for key in dict1:
    if key not in dict2:
        keys_to_drop.append(key)
keys_to_drop
for key in keys_to_drop:
    if key in dict1:
        del dict1[key]
len(dict1)

In [ ]:
dict1_list=list(dict1.values())
X2=[] #Xvalues_dict1=X2
for values in dict1_list:
    X2.append(values[0])
print(X2)

In [ ]:
dict1_list=list(dict1.values())
Y2=[]
for values in dict1_list:
    Y2.append(values[1])
print(Y2)

In [ ]:
dict2_list=list(dict2.values())
Y1=[]
for values in dict2_list:
    Y1.append(values[0][1])
print(Y1)

In [ ]:
dict2_list=list(dict2.values())
X1=[]
for values in dict2_list:
    X1.append(values[0][0])
print(X1)

### Calculating average offset, X-displacement, and Y-displacement

In [ ]:
changeinx=[]
changeiny=[]
r=[]
for x1, y1, x2, y2 in zip(X1,Y1,X2,Y2):
    delta_x = x2 - x1
    delta_y = y2 - y1
    distance= math.sqrt(delta_x**2 + delta_y**2)
    changeinx.append(delta_x)
    changeiny.append(delta_y)
    r.append(distance)

print(np.mean(r))
print(np.mean(changeinx))
print(np.mean(changeiny))

### Calculating average andgle of disoreintation of offsets with reference to center drawn using HoughCircle

In [ ]:
angles=[]
for delta_x,delta_y in zip(changeinx,changeiny):
    angle_in_radians = math.atan2(delta_y, delta_x)
    angle_in_degrees = math.degrees(angle_in_radians)
    angles.append(angle_in_degrees)
print(-np.mean(angles))

In [ ]:
Images=[]
for key in dict2:
    Images.append(key)
df1=pd.DataFrame({"Images":Images,"X1":X1,"Y1":Y1,"X2":X2,"Y2":Y2,"Delta_X":changeinx,"Delta_Y":changeiny,"r":r,"Angles":angles})
df1

In [ ]:
df2=df1.copy()
df2['ImageNumber']=df2['Images'].str.extract('(\d+)').astype(int)
\df2=df2.sort_values(by='ImageNumber')
df3=df2.copy()
df3=df3.reset_index(drop=True)
df2=df2.drop(columns=['ImageNumber'])
df2=df2.reset_index(drop=True)

In [ ]:
os.chdir("/home/abhinav/M35/") #Change the pathe here
dict3={}
for name in os.listdir():
    if ".txt" in name:
        filename=open(name,'r')
        lines=filename.readlines()
        (x,y)=((lines[0].split(",")[0]),(lines[0].split(",")[1]))
        point=(float(x),float(y))
        dict3[name]=[point[0],point[1]]
type(list(dict3.keys()))

In [ ]:
x=[]
y=[]
for values in list(dict3.values()):
    xvalue=values[0]
    yvalue=values[1]
    x.append(xvalue)
    y.append(yvalue)

In [ ]:
df4=pd.DataFrame({"Images":list(dict3.keys()),"X":x,"Y":y,})
df4['ImageNumber']=df4['Images'].str.extract('(\d+)').astype(int)
df4_sorted=df4.sort_values(by='ImageNumber')
df4_sorted=df4_sorted[df4_sorted['ImageNumber'].isin(df3['ImageNumber'])]
df4_sorted=df4_sorted.drop(columns=["ImageNumber"])
df4_sorted["Angles"]=df2["Angles"].values
df4_sorted = df4_sorted.reset_index(drop=True)

In [ ]:
finaldf=df4_sorted.copy()
finaldf["X1(Green dot in Pixels)"]=df2["X1"]
finaldf["X2(Blue dot in pixels)"]=df2["X2"]
finaldf["Y1(Green dot in pixels)"]=df2["Y1"]
finaldf["Y2(Blue dot in pixels)"]=df2["Y2"]
finaldf["Delta_x"]=df2["Delta_X"]
finaldf["Delta_y"]=df2["Delta_Y"]
finaldf["r"]=df2["r"]
finaldf


In [ ]:
x=np.array(finaldf['X'])
y=np.array(finaldf['Y'])
angles=np.array(finaldf['Angles'])
r=np.array(finaldf['r'])
print(np.mean(r))

In [ ]:
### Drawing arrowplots for the offset

In [ ]:
fig, ax = plt.subplots()

U = [r_value * math.cos(math.radians(-angle)) for angle,r_value in zip(angles, r)]
V = [r_value * math.sin(math.radians(-angle)) for angle,r_value in zip(angles,r)]

for i in range(len(U)):
    ax.quiver(x[i], y[i], U[i], V[i], scale=110 , color='b', width=0.008)

    
for i, label in enumerate(df3['ImageNumber']):
        plt.annotate(label, (x[i], y[i]),color='red', weight='bold', ha='center')
ax.set_xlim(-60,150)
ax.set_ylim(-20,240)
ax.set_xticks([])
ax.set_yticks([])

# Find the index for image with largest offset as reference
reference_arrow = finaldf[finaldf['r']== finaldf['r'].max()].index[0]

# Calculate arrow length for image with largest offset
arrow_length = math.sqrt(U[reference_arrow]**2 + V[reference_arrow]**2)

ax.quiver(x[reference_arrow], y[reference_arrow], U[reference_arrow], V[reference_arrow], scale=110, color='g', width=0.008)

bottom_left_x, bottom_left_y = -40, 180  # Adjust these as needed
ax.quiver(bottom_left_x, bottom_left_y, arrow_length, 0, scale=110, color='g', width=0.008)

plt.annotate(f'Length: {(arrow_length)*2/103:.2f}mm', (bottom_left_x + arrow_length, bottom_left_y+5), color='red', fontsize=8, weight='bold',ha='center')

ax.set_title('Combined Arrowplots')
ax.grid(True)
plt.tight_layout()

